#朴素贝叶斯

In [6]:
import os 
import numpy as np

In [7]:
#朴素贝叶斯分类器
class NBayes():
    #设置属性
    def __init__(self) -> None:
        self.trainSet=0  #训练集
        self.trainLabel=0 #训练集标签
        self.yProba={}#先验概率容器
        self.xyProba={}#条件概率容器
        self.ySet={} #标记类别对应的数量
        self.ls=1    #拉普拉斯平滑系数
        self.n_samples=0 #训练集样本数量
        self.n_features=0 #训练集特征数量
    
    #计算先验概率——每个标记的占比
    def comPy(self,y,Ls=True):
        Py={}
        yi={}
        #获取类别数
        ySet=np.unique(y)
        #带入拉普拉斯平滑做统计处理
        for i in ySet:
            Py[i]=(sum(y==i)+self.ls)/(self.n_samples+len(ySet))#Py:{y1:a,y2:b} a+b=1
            yi[i]=sum(y==i)#yi={y1:m,y2:n} m+n=y.len()
        self.yProba=Py
        self.ySet=yi
        return
    
    #计算条件概率
    def comPxy(self,X,y,LS=True):
        Pxy={}
        for yi,yiCount in self.ySet.items():#(yi,yiCount)=》y的一个类别，这个类别占的数量
            Pxy[yi]={}#Pxy={y1:{},y2:{}}
            for xindex in range(self.n_features):#遍历访问特征矩阵的每一个特征Xi 非具体的值  而是特征名称
                Pxy[yi][xindex]={}#Pxy=Pxy={y1:{X1:{},X2:{}....},y2:{}}
                #下标为第index的特征数据
                Xi=X[:,xindex]#获取第xindex个特征整列的数据
                Xiset=np.unique(Xi)
                Xicount=Xiset.size#获取这一列也就是Xi这个特征下的取值类别和类别数
                #下标为index，且标记为yi的特征数据
                Xiyi=X[np.nonzero(y==yi)[0],xindex]#nonzero（）用于得到数组中非零元素的位置（数组索引）
                #Xiyi是一个数组 记录的是y=yi的条件下且位于index列的特征数据
                for xi in Xiset:
                    Pxy[yi][xindex][xi] = self.classifyProba(xi, Xiyi, Xicount)    #第三层是变量Xi的分类概率，离散变量 实际上就是p（XI=xi|yi）的条件概率
        self.xyProba = Pxy
        return        
    #离散变量直接计算概率
    def classifyProba(self, x, xArr, Xicount):
        Pxy = (sum(xArr == x) + self.ls) / (xArr.size + Xicount)    #加入拉普拉斯修正的概率
        return Pxy
                    

    #训练
    def train(self,X,y):
        self.n_samples,self.n_features=X.shape
        #计算先验概率  就是不同类的y所占比例和数量
        self.comPy(y)
        print('P(y)计算完毕')
        #计算条件概率
        self.comPxy(X,y)
        print('P(x|y)计算完毕')
        self.trainSet=X
        self.trainLabel=y
        return
    

    #预测
    def predict(self,X):
        m,n=X.shape
        #生成一个维度是((m,len(self.yProba))的二维数组
        proba=np.zeros((m,len(self.yProba)))#proba=[[0,0],[0,0],....]
        for i in range(m):
            for idx,(yi,Py) in enumerate(self.yProba.items()):
                #0 (0.0, 0.5263157894736842)
                #1 (1.0, 0.47368421052631576)
                proba_idx=Py
                for xidx in range(n):
                    xi=X[i,xidx]
                    proba_idx*=self.xyProba[yi][xidx][xi]#计算的是p（yi）*(p（X1=xi|yi）*p（X2=xi|yi）*....
                proba[i,idx]=proba_idx
        return proba
    



In [8]:

from sklearn.preprocessing import OrdinalEncoder
dataSet = [
        ['青绿', '蜷缩', '浊响', '清晰', '凹陷', '硬滑', 0.697, 0.460, '好瓜'],
        ['乌黑', '蜷缩', '沉闷', '清晰', '凹陷', '硬滑', 0.774, 0.376, '好瓜'],
        ['乌黑', '蜷缩', '浊响', '清晰', '凹陷', '硬滑', 0.634, 0.264, '好瓜'],
        ['青绿', '蜷缩', '沉闷', '清晰', '凹陷', '硬滑', 0.608, 0.318, '好瓜'],
        ['浅白', '蜷缩', '浊响', '清晰', '凹陷', '硬滑', 0.556, 0.215, '好瓜'],
        ['青绿', '稍蜷', '浊响', '清晰', '稍凹', '软粘', 0.403, 0.237, '好瓜'],
        ['乌黑', '稍蜷', '浊响', '稍糊', '稍凹', '软粘', 0.481, 0.149, '好瓜'],
        ['乌黑', '稍蜷', '浊响', '清晰', '稍凹', '硬滑', 0.437, 0.211, '好瓜'],
        ['乌黑', '稍蜷', '沉闷', '稍糊', '稍凹', '硬滑', 0.666, 0.091, '坏瓜'],
        ['青绿', '硬挺', '清脆', '清晰', '平坦', '软粘', 0.243, 0.267, '坏瓜'],
        ['浅白', '硬挺', '清脆', '模糊', '平坦', '硬滑', 0.245, 0.057, '坏瓜'],
        ['浅白', '蜷缩', '浊响', '模糊', '平坦', '软粘', 0.343, 0.099, '坏瓜'],
        ['青绿', '稍蜷', '浊响', '稍糊', '凹陷', '硬滑', 0.639, 0.161, '坏瓜'],
        ['浅白', '稍蜷', '沉闷', '稍糊', '凹陷', '硬滑', 0.657, 0.198, '坏瓜'],
        ['乌黑', '稍蜷', '浊响', '清晰', '稍凹', '软粘', 0.360, 0.370, '坏瓜'],
        ['浅白', '蜷缩', '浊响', '模糊', '平坦', '硬滑', 0.593, 0.042, '坏瓜'],
        ['青绿', '蜷缩', '沉闷', '稍糊', '稍凹', '硬滑', 0.719, 0.103, '坏瓜']
    ]
#特征值列表
labels = ['色泽', '根蒂', '敲击', '纹理', '脐部', '触感', '密度', '含糖率']
dataX = np.array(dataSet)[:,:6]
oriencode = OrdinalEncoder(categories='auto')
oriencode.fit(dataX)
X=oriencode.transform(dataX)
y = np.array(dataSet)[:,8]
y[y=="好瓜"]=1
y[y=="坏瓜"]=0
y=y.astype(float)
#训练
NB = NBayes()
NB.train(X, y)
Proba = NB.predict(X)
#返回np数组中最大值的索引
yPredict = np.argmax(Proba, axis=1)
sum((y-yPredict)==0)

P(y)计算完毕
P(x|y)计算完毕


14

In [9]:

#打印先验概率
print(NB.yProba)
print('\n')
#打印条件概率
print(NB.xyProba)

{0.0: 0.5263157894736842, 1.0: 0.47368421052631576}


{0.0: {0: {0.0: 0.25, 1.0: 0.4166666666666667, 2.0: 0.3333333333333333}, 1: {0.0: 0.25, 1.0: 0.4166666666666667, 2.0: 0.3333333333333333}, 2: {0.0: 0.3333333333333333, 1.0: 0.4166666666666667, 2.0: 0.25}, 3: {0.0: 0.3333333333333333, 1.0: 0.25, 2.0: 0.4166666666666667}, 4: {0.0: 0.25, 1.0: 0.4166666666666667, 2.0: 0.3333333333333333}, 5: {0.0: 0.6363636363636364, 1.0: 0.36363636363636365}}, 1.0: {0: {0.0: 0.45454545454545453, 1.0: 0.18181818181818182, 2.0: 0.36363636363636365}, 1: {0.0: 0.09090909090909091, 1.0: 0.36363636363636365, 2.0: 0.5454545454545454}, 2: {0.0: 0.2727272727272727, 1.0: 0.6363636363636364, 2.0: 0.09090909090909091}, 3: {0.0: 0.09090909090909091, 1.0: 0.7272727272727273, 2.0: 0.18181818181818182}, 4: {0.0: 0.5454545454545454, 1.0: 0.09090909090909091, 2.0: 0.36363636363636365}, 5: {0.0: 0.7, 1.0: 0.3}}}


In [10]:
yPredict

array([1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0], dtype=int64)

#朴素贝叶斯——连续值处理


In [11]:
# 准备数据
import math
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
class Bayes(object):
    def __init__(self,trainData):
        self.trainData=trainData
        # model_par以字典形式存放每一个类别的方差
        self.model_para={}#均值和方差
        self.yproba=0#先验概率

        
    def tarin_bayesModel(self):
        # 将训练集按照类别进行提取
        Py={}
        #获取类别数
        ySet=np.unique(pd.DataFrame(self.trainData).iloc[:,-1])
        #带入拉普拉斯平滑做统计处理
        for i in ySet:
            Py[i]=(sum(pd.DataFrame(self.trainData).iloc[:,-1]==i)+1)/(len(pd.DataFrame(self.trainData).iloc[:,-1])+len(ySet))#Py:{y1:a,y2:b} a+b=1
        self.yProba=Py
   
       
        separated_class=self.separateByClass()#{0:[v1,v2,v3....],1:{v5,v7..}...}
        # vectors是列表，包含的是每个类别对应的向量集
        for classValue, vectors in separated_class.items():
            #例如1，[v5,v7] vi是向量
            # 将每一个类别的均值和方差保存在对应的键值对中
            self.model_para[classValue] = self.summarize(vectors)#self.model_para={0:[(mean1,variance1),(mean2,variance2)...],1:[..]..}
        return self.model_para
    
    # 计算均值
    def mean(self,numbers):
        return sum(numbers) / float(len(numbers))
 
    # 计算方差，注意是分母是n-1，自由度
    def stdev(self,numbers):
        avg = self.mean(numbers)
        variance = sum([pow(x - avg, 2) for x in numbers]) / float(len(numbers) - 1)
        return math.sqrt(variance)
 
    # 对每一类样本的每个特征计算均值和方差，结果保存在列表中，依次为第一维特征、第二维特征等...的均值和方差
    def summarize(self,vectors):
        # zip利用 * 号操作符，可以将不同元组或者列表压缩为为列表集合。用来提取每类样本下的每一维的特征集合
        summaries = [(self.mean(attribute), self.stdev(attribute)) for attribute in zip(*vectors)]
        #zip(*[[4.9, 3. , 1.4, 0.2],[4.7, 3.2, 1.3, 0.2]])=>[(4.9,4.7),(3,3.2)...]即attitude是这个类别下某一列特征的值的集合
        # 将代表类别的最后一个数据删掉，只保留均值和方差
        del summaries[-1]
        return summaries
 
 
    # 将训练集按照类别进行提取，以字典形式存放，Key为类别，value为列表，列表中包含的是每个类别对应的向量集
    def separateByClass(self):
        #字典用于存放分类后的向量集合
        separated_class = {}
        for i in range(len(self.trainData)):
            vector = self.trainData[i]
            # vector[-1]为每组数据的类别
            if (vector[-1] not in separated_class):
                separated_class[vector[-1]] = []
            #     将每列数据存放在对应的类别下，列表形式
            separated_class[vector[-1]].append(vector)
        return separated_class
 
    # 假定服从正态分布，对连续属性计算概率密度函数,公式参考周志华老师的西瓜书P151
    def calProbabilityDensity(self,x, mean, stdev):
        # x为待分类数据
        exponent = math.exp(-(math.pow(x-mean,2)/(2*math.pow(stdev,2))))
        return (1 / (math.sqrt(2*math.pi) * stdev)) * exponent
 
    # 计算待分类数据的联合概率
    def calClassProbabilities(self, inputVector):
        # summaries为训练好的贝叶斯模型参数, inputVector为待分类数据(单个)
        # probabilities用来保存待分类数据对每种类别的联合概率
        probabilities = {}
        # classValue为字典的key(类别) ,classSummaries为字典的vlaue(每个类别每维特征的均值和方差),列表形式
        for classValue, classSummaries in self.model_para.items():
            probabilities[classValue] = self.yProba[classValue]#获得p（c）
            # len(classSummaries)表示有多少特征维度
            for i in range(len(classSummaries)):
                # mean, stdev分贝别表示每维特征对应的均值和方差
                mean, stdev = classSummaries[i]
                # 提取待分类数据的i维数据值
                x = inputVector[i]
                # 计算联合概率密度
                probabilities[classValue] *= self.calProbabilityDensity(x, mean, stdev)
        # 返回概率最大的类别
        prediction=max(probabilities,key=probabilities.get)
        return prediction

In [12]:
# 计算分类准确率
def calAccuracy(testData,bayes):
    correct_nums=0
    for i in range(len(testData)):
        # 逐次计算每一个数据的分类类别
        if  testData[i][-1]== bayes.calClassProbabilities(testData[i]):
            correct_nums += 1
    return correct_nums
 
d=load_iris()
train_data=pd.DataFrame(d['data'])
train_target=pd.DataFrame(d['target'])
X_train, X_test, y_train, y_test = train_test_split(train_data, train_target, test_size=0.3, random_state=42)
trainData=pd.concat([X_train,y_train],axis=1)
bayes=Bayes(np.array(trainData))
# model为训练之后的bayes分类器模型的概率参数
model=bayes.tarin_bayesModel()
testData=pd.concat([X_test,y_test],axis=1)
correct_nums=calAccuracy(np.array(testData), bayes)
print("分类准确率 %f"%(correct_nums/len(testData) * 100.0))

分类准确率 97.777778
